In [15]:
import cobra
model = cobra.io.read_sbml_model('/data1/xpgeng/cross_pathogen/GEM_model/iPC815.xml')

Scaling...
 A: min|aij| =  1.000e+00  max|aij| =  1.000e+00  ratio =  1.000e+00
Problem data seem to be well scaled


In [1]:
import csv
one_gene = {}
with open('/data1/xpgeng/cross_pathogen/FBA/' +'iPC815_1KO_growth', 'r') as csvfile:
    reader = csv.reader(csvfile)
    next(reader) 
    for row in reader:
        #print(len(row))
        if len(row) == 4:            
            one_gene[row[0]] = int(row[3])           
    print(len(one_gene)) 

814


In [2]:
gene_list = []
for k,v in one_gene.items():
    gene_list.append(k)
print(len(gene_list))

814


In [3]:
import csv
from itertools import combinations
from tqdm import tqdm
import cobra.flux_analysis
from multiprocessing import Pool

# Adjust the function to use model and gene_list as arguments
def knockout(gene, gene_list, model):
    rest_list = [g for g in gene_list if g != gene]
    twogene = list(combinations(rest_list, 2))
    
    filepath = f'iPC_815_{gene}.csv'

    with open('/data1/xpgeng/cross_pathogen/FBA/iPC815-3KO/' + filepath, mode='a', newline='') as csvfile:
        writer = csv.writer(csvfile)
        writer.writerow(['Gene', 'Gene1', 'Gene2', 'Initial Growth', 'Altered Growth', 'Label'])

        initial_state = cobra.flux_analysis.pfba(model)
        initial_growth = round(initial_state.fluxes['BIOMASS_37C'], 3)

        for gene1, gene2 in tqdm(twogene):
            with model:
                try:
                    model.genes.get_by_id(gene).knock_out()
                    model.genes.get_by_id(gene1).knock_out()
                    model.genes.get_by_id(gene2).knock_out()
                    altered_state = cobra.flux_analysis.pfba(model)
                    altered_growth = round(altered_state.fluxes['BIOMASS_37C'], 3)
                    label = 1 if altered_growth < 0.1 * initial_growth else 0
                    writer.writerow([gene, gene1, gene2, initial_growth, altered_growth, label])
                except Exception as e:
                    writer.writerow([gene, gene1, gene2, 'Failed', str(e)])

In [ ]:
def run_knockouts(gene_list, model):
    # Slice the first 100 genes from the list
    top_genes = gene_list[:100]
    
    # Create a multiprocessing pool with 100 workers
    with Pool(100) as pool:
        # Each pool worker will execute the `knockout` function with a gene, the full gene list, and the model
        results = pool.starmap(knockout, [(gene, gene_list, model) for gene in top_genes])

if __name__ == "__main__":
    run_knockouts(gene_list, model)

  1%|▌                                                                                            | 1850/330078 [13:52<47:26:16,  1.92it/s]

In [ ]:
def run_knockouts(gene_list, model):
    # Slice the first 100 genes from the list
    top_genes = gene_list[100:200]
    
    # Create a multiprocessing pool with 100 workers
    with Pool(100) as pool:
        # Each pool worker will execute the `knockout` function with a gene, the full gene list, and the model
        results = pool.starmap(knockout, [(gene, gene_list, model) for gene in top_genes])

if __name__ == "__main__":
    run_knockouts(gene_list, model)

  0%|                                   | 350/330078 [01:29<20:48:14,  4.40it/s]

In [ ]:
def run_knockouts(gene_list, model):
    # Slice the first 100 genes from the list
    top_genes = gene_list[200:300]
    
    # Create a multiprocessing pool with 100 workers
    with Pool(100) as pool:
        # Each pool worker will execute the `knockout` function with a gene, the full gene list, and the model
        results = pool.starmap(knockout, [(gene, gene_list, model) for gene in top_genes])

if __name__ == "__main__":
    run_knockouts(gene_list, model)

  0%|                                   | 135/330078 [00:55<39:51:01,  2.30it/s]

In [ ]:
def run_knockouts(gene_list, model):
    # Slice the first 100 genes from the list
    top_genes = gene_list[300:400]
    
    # Create a multiprocessing pool with 100 workers
    with Pool(100) as pool:
        # Each pool worker will execute the `knockout` function with a gene, the full gene list, and the model
        results = pool.starmap(knockout, [(gene, gene_list, model) for gene in top_genes])

if __name__ == "__main__":
    run_knockouts(gene_list, model)

  0%|                          | 18/330078 [00:05<24:05:20,  3.81it/s]

In [ ]:
def run_knockouts(gene_list, model):
    # Slice the first 100 genes from the list
    top_genes = gene_list[700:]
    
    # Create a multiprocessing pool with 100 workers
    with Pool(100) as pool:
        # Each pool worker will execute the `knockout` function with a gene, the full gene list, and the model
        results = pool.starmap(knockout, [(gene, gene_list, model) for gene in top_genes])

if __name__ == "__main__":
    run_knockouts(gene_list, model)

  1%|▏                       | 1888/330078 [08:37<23:55:22,  3.81it/s]

due to YPO3097 missing some data, we rerun it \
but some data still missing.

In [ ]:
def run_knockouts(gene_list, model):
    # Slice the first 100 genes from the list
    top_genes = ['YPO3097']
    
    # Create a multiprocessing pool with 100 workers
    with Pool(1) as pool:
        # Each pool worker will execute the `knockout` function with a gene, the full gene list, and the model
        results = pool.starmap(knockout, [(gene, gene_list, model) for gene in top_genes])

if __name__ == "__main__":
    run_knockouts(gene_list, model)

 93%|██████████████████▌ | 305692/330078 [32:37:08<1:14:19,  5.47it/s]

In [21]:
import pandas as pd
import os
from tqdm import tqdm  # 如果需要进度条

folder_path = '/data1/xpgeng/cross_pathogen/FBA/iPC815-3KO/'
csv_files = [f for f in os.listdir(folder_path) if f.endswith('.csv')]

df_list = []
for file in tqdm(csv_files, desc="Reading CSV files"):
    file_path = os.path.join(folder_path, file)
    try:
        df = pd.read_csv(file_path)
        df_list.append(df)
    except Exception as e:
        print(f"Error reading {file_path}: {e}")

if df_list:
    combined_df = pd.concat(df_list, ignore_index=True)
    output_path = 'combined_iPC815_output.csv'
    combined_df.to_csv(output_path, index=False)
    print(f"Combined file written to {output_path}")
else:
    print("No data to combine.")

Reading CSV files:   0%|                      | 0/814 [00:00<?, ?it/s]/tmp/ipykernel_13691/1791842788.py:12: DtypeWarning: Columns (3,4) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file_path)
Reading CSV files:   0%|              | 1/814 [00:00<09:04,  1.49it/s]/tmp/ipykernel_13691/1791842788.py:12: DtypeWarning: Columns (3,4) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file_path)
Reading CSV files:   0%|              | 2/814 [00:01<08:21,  1.62it/s]/tmp/ipykernel_13691/1791842788.py:12: DtypeWarning: Columns (3,4) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file_path)
Reading CSV files:   0%|              | 3/814 [00:01<08:00,  1.69it/s]/tmp/ipykernel_13691/1791842788.py:12: DtypeWarning: Columns (3,4) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file_path)
Reading CSV files:   1%|            

Reading CSV files:  12%|█▌           | 98/814 [01:03<06:49,  1.75it/s]/tmp/ipykernel_13691/1791842788.py:12: DtypeWarning: Columns (3,4) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file_path)
Reading CSV files:  12%|█▌           | 99/814 [01:05<11:03,  1.08it/s]/tmp/ipykernel_13691/1791842788.py:12: DtypeWarning: Columns (3,4) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file_path)
Reading CSV files:  13%|█▌          | 107/814 [01:12<11:11,  1.05it/s]/tmp/ipykernel_13691/1791842788.py:12: DtypeWarning: Columns (3,4) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file_path)
Reading CSV files:  14%|█▌          | 110/814 [01:13<08:28,  1.38it/s]/tmp/ipykernel_13691/1791842788.py:12: DtypeWarning: Columns (3,4) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file_path)
Reading CSV files:  15%|█▊          

Reading CSV files:  26%|███         | 208/814 [02:13<05:34,  1.81it/s]/tmp/ipykernel_13691/1791842788.py:12: DtypeWarning: Columns (3,4) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file_path)
Reading CSV files:  26%|███         | 209/814 [02:14<05:27,  1.85it/s]/tmp/ipykernel_13691/1791842788.py:12: DtypeWarning: Columns (3,4) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file_path)
Reading CSV files:  26%|███         | 211/814 [02:16<08:32,  1.18it/s]/tmp/ipykernel_13691/1791842788.py:12: DtypeWarning: Columns (3,4) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file_path)
Reading CSV files:  26%|███▏        | 212/814 [02:17<07:33,  1.33it/s]/tmp/ipykernel_13691/1791842788.py:12: DtypeWarning: Columns (3,4) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file_path)
Reading CSV files:  28%|███▍        

Reading CSV files:  40%|████▊       | 328/814 [03:27<05:21,  1.51it/s]/tmp/ipykernel_13691/1791842788.py:12: DtypeWarning: Columns (3,4) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file_path)
Reading CSV files:  41%|████▊       | 330/814 [03:28<04:44,  1.70it/s]/tmp/ipykernel_13691/1791842788.py:12: DtypeWarning: Columns (3,4) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file_path)
Reading CSV files:  41%|████▉       | 331/814 [03:28<04:32,  1.77it/s]/tmp/ipykernel_13691/1791842788.py:12: DtypeWarning: Columns (3,4) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file_path)
Reading CSV files:  41%|████▉       | 332/814 [03:29<04:27,  1.80it/s]/tmp/ipykernel_13691/1791842788.py:12: DtypeWarning: Columns (3,4) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file_path)
Reading CSV files:  41%|████▉       

Reading CSV files:  51%|██████      | 412/814 [04:16<04:25,  1.52it/s]/tmp/ipykernel_13691/1791842788.py:12: DtypeWarning: Columns (3,4) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file_path)
Reading CSV files:  51%|██████      | 414/814 [04:17<04:03,  1.64it/s]/tmp/ipykernel_13691/1791842788.py:12: DtypeWarning: Columns (3,4) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file_path)
Reading CSV files:  51%|██████      | 415/814 [04:18<04:23,  1.52it/s]/tmp/ipykernel_13691/1791842788.py:12: DtypeWarning: Columns (3,4) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file_path)
Reading CSV files:  52%|██████▏     | 420/814 [04:22<04:07,  1.59it/s]/tmp/ipykernel_13691/1791842788.py:12: DtypeWarning: Columns (3,4) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file_path)
Reading CSV files:  52%|██████▏     

Reading CSV files:  63%|███████▌    | 509/814 [05:16<03:12,  1.58it/s]/tmp/ipykernel_13691/1791842788.py:12: DtypeWarning: Columns (3,4) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file_path)
Reading CSV files:  63%|███████▌    | 510/814 [05:16<03:03,  1.65it/s]/tmp/ipykernel_13691/1791842788.py:12: DtypeWarning: Columns (3,4) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file_path)
Reading CSV files:  63%|███████▌    | 512/814 [05:17<02:51,  1.76it/s]/tmp/ipykernel_13691/1791842788.py:12: DtypeWarning: Columns (3,4) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file_path)
Reading CSV files:  63%|███████▌    | 513/814 [05:19<04:07,  1.22it/s]/tmp/ipykernel_13691/1791842788.py:12: DtypeWarning: Columns (3,4) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file_path)
Reading CSV files:  63%|███████▌    

Reading CSV files:  73%|████████▊   | 595/814 [06:14<02:23,  1.52it/s]/tmp/ipykernel_13691/1791842788.py:12: DtypeWarning: Columns (3,4) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file_path)
Reading CSV files:  73%|████████▊   | 596/814 [06:14<02:14,  1.62it/s]/tmp/ipykernel_13691/1791842788.py:12: DtypeWarning: Columns (3,4) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file_path)
Reading CSV files:  73%|████████▊   | 598/814 [06:17<03:48,  1.06s/it]/tmp/ipykernel_13691/1791842788.py:12: DtypeWarning: Columns (3,4) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file_path)
Reading CSV files:  74%|████████▊   | 599/814 [06:17<03:13,  1.11it/s]/tmp/ipykernel_13691/1791842788.py:12: DtypeWarning: Columns (3,4) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file_path)
Reading CSV files:  74%|████████▊   

Reading CSV files:  83%|█████████▉  | 676/814 [07:08<01:19,  1.74it/s]/tmp/ipykernel_13691/1791842788.py:12: DtypeWarning: Columns (3,4) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file_path)
Reading CSV files:  83%|█████████▉  | 678/814 [07:09<01:37,  1.39it/s]/tmp/ipykernel_13691/1791842788.py:12: DtypeWarning: Columns (3,4) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file_path)
Reading CSV files:  84%|██████████  | 684/814 [07:14<01:19,  1.63it/s]/tmp/ipykernel_13691/1791842788.py:12: DtypeWarning: Columns (3,4) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file_path)
Reading CSV files:  84%|██████████  | 685/814 [07:15<01:50,  1.17it/s]/tmp/ipykernel_13691/1791842788.py:12: DtypeWarning: Columns (3,4) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file_path)
Reading CSV files:  84%|██████████  

Reading CSV files:  97%|███████████▋| 793/814 [08:23<00:11,  1.87it/s]/tmp/ipykernel_13691/1791842788.py:12: DtypeWarning: Columns (3,4) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file_path)
Reading CSV files:  98%|███████████▋| 794/814 [08:23<00:10,  1.92it/s]/tmp/ipykernel_13691/1791842788.py:12: DtypeWarning: Columns (3,4) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file_path)
Reading CSV files:  98%|███████████▋| 795/814 [08:24<00:10,  1.89it/s]/tmp/ipykernel_13691/1791842788.py:12: DtypeWarning: Columns (3,4) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file_path)
Reading CSV files:  99%|███████████▉| 806/814 [08:31<00:04,  1.63it/s]/tmp/ipykernel_13691/1791842788.py:12: DtypeWarning: Columns (3,4) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file_path)
Reading CSV files: 100%|███████████▉

Combined file written to combined_iPC815_output.csv


In [4]:
import csv
three_gene = {}
with open('/data1/xpgeng/cross_pathogen/FBA/' +'combined_iPC815_output.csv', 'r') as csvfile:
    reader = csv.reader(csvfile)
    next(reader) 
    for row in reader:
        #print(len(row))
        try:
            
            if len(row) == 6 and 'Gene' not in row and 'Failed' not in row:            
                three_gene[row[0], row[1], row[2]] = float(row[5])  
        except:
            print(row)
    print(len(three_gene)) 

268653502


In [5]:
# merge dedundant elements

sorted_key_dict = {}

for key, value in tqdm(three_gene.items()):
    # 对键进行排序
    sorted_key = tuple(sorted(key))
    
    # 将排序后的键和值存入新字典
    sorted_key_dict[sorted_key] = value

print(len(sorted_key_dict))

100%|███████████████| 268653502/268653502 [11:30<00:00, 389054.09it/s]

89560678


In [6]:
814*813*812/6

89561164.0

In [24]:

count = 0
for k, value in sorted_key_dict.items():
    if value == 1.0:
        count += 1


print(count)  

39936493


### Features Processing

In [7]:
# protein =============================================================

from tqdm import tqdm

from Bio import SeqIO

path = '/data1/xpgeng/cross_pathogen/fasta/'
# calculate features 2-mer
amino_acids = 'ACDEFGHIKLMNPQRSTVWY'
all_2_mers = [a+b for a in amino_acids for b in amino_acids]

pepdic = {}
for r in SeqIO.parse(path + 'iPC815_protein.fasta', 'fasta'):     
    pepdic[r.id] = str(r.seq)
print(len(pepdic))

two_mer_feature_dic = {}
for gene, sequ in pepdic.items():
    feature_vector = {two_mer: 0 for two_mer in all_2_mers}  
    for i in range(len(sequ) - 1):
        mer = sequ[i:i+2]
        if mer in feature_vector:
            feature_vector[mer] += 1   
    two_mer_feature_dic[gene] = [feature_vector[mer] for mer in all_2_mers]


814


### Train_test Split

YPO1222 one ko fail, it is related too many reactions, delete it  \
YPO2264 has no protein sequence, add it from Uniprot, Q8ZEB6

In [8]:
import numpy as np
from tqdm import tqdm

three_gene_items = list(sorted_key_dict.items())

np.random.shuffle(three_gene_items)

In [26]:
ss = len(three_gene_items) // 10

In [29]:
ss

8956067

to fully use all data, we divide data into 10 parts, each of them has 9,000,0000 elements.

In [9]:
for i in range(20):
    sampled_three_gene = three_gene_items[i*4500000: (i+1)*4500000]

    feature_list = []
    label_list = []

    for key, value in tqdm(sampled_three_gene):
        try:

            features = [a + b + c for a, b, c in zip(two_mer_feature_dic[key[0]],
                                                 two_mer_feature_dic[key[1]],
                                                 two_mer_feature_dic[key[2]])]

            feature_list.append(features)
            label_list.append(value)
        except:
            print(key)

    X = np.array(feature_list)
    y = np.array(label_list)

    import pickle
    with open(f'/data2/xpgeng/iPC815/X_{i}.pickle', 'wb') as f:
        pickle.dump(X, f)

    with open(f'/data2/xpgeng/iPC815/y_{i}.pickle', 'wb') as f:
        pickle.dump(y, f)

100%|█████████████████████| 4060678/4060678 [07:45<00:00, 8718.80it/s]
